## Problem Statement
Find the minimum of $f(x) =\left(\frac{\sqrt{3}}{2} (x_1−3)+\frac{1}{2}(x_2−5)\right)^2+ 5 \cdot \left((\frac{\sqrt{3}}{2} (x_2−5)−\frac{1}{2} (x_1−3)\right)^2$ using GA.


In [7]:
import numpy as np
import math
def generate_population(param_size, pop_size, low_bound, high_bound, discrete = False):
    """
    Generates a random matrix of size (pop_size, param_size) with values in the given bounds. 
    Integer values are used if discrete = True
    """
    population = np.random.uniform(low= low_bound, high = high_bound, size=(pop_size, param_size))
    if discrete:
        population = np.round(population)
    return population

In [8]:
PARAM_SIZE = 2
POP_SIZE = 10
LOW_BOUND = -10
HIGH_BOUND = 10
population = generate_population(PARAM_SIZE, POP_SIZE, LOW_BOUND, HIGH_BOUND)
print(population.shape)
print(np.mean(population))

(10, 2)
-1.4068488937349861


In [9]:
def calculate_fitness(total_pop):
    """ Calculates fitness using the given function f(x) = 10 *x^2
    Returns a (POP_SIZE, ) numpy array
    """       
    fitness = np.zeros(total_pop.shape[0])
    for i in range(total_pop.shape[0]):
        x_1, x_2 = total_pop[i]
        factor1 = (math.sqrt(3)/2 *(x_1-3) + 0.5*(x_2-5))**2
        factor2 = 5*(math.sqrt(3)/2 *(x_2-5) - 0.5*(x_1-3))**2
        fitness[i] = factor1+ factor2
    return fitness

In [10]:
fitness = calculate_fitness(population)
print(fitness.shape)
print(fitness)

(10,)
[128.22496903   5.30161088 147.60348891 218.80873982 144.76880972
 513.94991694 184.84119576 331.51793573 559.57729979  83.15654571]


In [11]:
def select_determinstic(t_pop, t_fitness, n_mating, n_params):
    """ Given current population t_pop, select n_mating number of parents using determinstic selection scheme
    based on t_fitness (the fitness of the population) 

    Returns parents and their fitness as a tuple (parent, fitness_of_parents)
    """    
    parents = np.empty((n_mating, n_params))
    
    best_fitness = np.argsort(t_fitness)[0:n_mating]
    parents = t_pop[best_fitness]
    return (parents, calculate_fitness(parents))

In [12]:
N_MATING = 4
parents, par_fitness = select_determinstic(population, fitness, N_MATING, PARAM_SIZE)
print(parents.shape)
print(parents)
print(par_fitness)

(4, 2)
[[ 1.87849565  3.54879218]
 [-0.28750032  7.74920869]
 [-4.82781592 -1.98230003]
 [ 3.2484462  -0.90681181]]
[  5.30161088  83.15654571 128.22496903 144.76880972]


In [13]:
def crossover(t_parents, n_offspring, idx_crossover):
    """ Given a set of parents, combine them and return offspring vectors
    
    Returns the offsprings and their fitness
    """
    
    # Create an emppty vector
    offspring = np.empty((n_offspring, t_parents.shape[1]))

    # Fill in crossover details
    for i in range(n_offspring):
        parent1 = t_parents[i]
        parent2 = t_parents[i+1]
        offspring[i] = np.copy(parent2)
        offspring[i,idx_crossover:] = parent1[idx_crossover:]
        
    return (offspring, calculate_fitness(offspring))
def mutation(t_offspring, mutation_rate):
    """ Given a set of offsprings, introduce mutation in them
   
    Returns the mutated offsprings and their fitness
    """
    # Fill in details
    mutated_offspring = np.copy(t_offspring)
    
    random_values = np.random.uniform(low=-0.5, high=0.5, size=t_offspring.shape)
    coin_toss = np.random.uniform(low=0, high=1, size=t_offspring.shape)
    ind = np.nonzero(coin_toss > mutation_rate)
    print(ind)
    print(random_values.shape)
    mutated_offspring[ind] += random_values[ind]
        
    return (mutated_offspring, calculate_fitness(mutated_offspring))

In [14]:
N_OFFSPRING =  N_MATING-1 # Fill in how many offspring you want
IDX_CROSSOVER = 0 # Fill in at which index you want crossover
PM = 0.5 # Mutation rate parameter

offspring,  offspring_fitness = crossover(parents, N_OFFSPRING, IDX_CROSSOVER)
offspring, offspring_fitness = mutation(offspring, 0.5)
print(offspring)
print(offspring_fitness)

(array([], dtype=int64), array([], dtype=int64))
(3, 2)
[[ 1.87849565  3.54879218]
 [-0.28750032  7.74920869]
 [-4.82781592 -1.98230003]]
[  5.30161088  83.15654571 128.22496903]


In [15]:
def environmental_selection(curr_population, offspring):
    """ Calculate total population (after constraint checking) fitness,
    rank accoridingly and select only the top POP_SIZE individuals to
    pass on to the next generation
    """ 

    # Fill in details
    t_total_pop = np.vstack((curr_population, offspring))
    new_pop, new_pop_fitness = select_determinstic(t_total_pop, calculate_fitness(t_total_pop), curr_population.shape[0], curr_population.shape[1])
    return new_pop

In [16]:
new_pop = environmental_selection(population, offspring)
print(new_pop)

[[ 1.87849565  3.54879218]
 [ 1.87849565  3.54879218]
 [-0.28750032  7.74920869]
 [-0.28750032  7.74920869]
 [-4.82781592 -1.98230003]
 [-4.82781592 -1.98230003]
 [ 3.2484462  -0.90681181]
 [-6.58422685  3.71267477]
 [-6.67348527 -3.30792878]
 [-6.96681731 -4.17983422]]


In [17]:
best_outputs = []
num_generations = 1000
curr_population = generate_population(PARAM_SIZE, POP_SIZE, LOW_BOUND, HIGH_BOUND)
overall_max_fitness = 99999

# Run many iterations
# You should also have another convergence check
for generation in range(num_generations):
    print("Generation : ", generation)

    # Measuring the fitness of each chromosome in the population.
    fitness = calculate_fitness(curr_population)
    print('Fitness')
    print(fitness.shape)
    print(fitness.T)

    min_fitness = np.min(fitness)

    # The best result in the current iteration.
    print("Best result in current iteration {0} compared to overall {1}".format(min_fitness, min(min_fitness, overall_max_fitness)))
    best_outputs.append(min_fitness)
    
    # Selecting the best parents in the population for mating.
    N_MATING = 6
    parents, par_fitness = select_determinstic(curr_population, fitness, N_MATING, PARAM_SIZE)
    print("Parents")
    print(parents.shape)
    print(parents)
    print(par_fitness)
#     parents, _ = select_stochastic(curr_population, fitness)
    
    # print("Parents")
    # print(parents)
    N_OFFSPRING =  N_MATING-1 # Fill in how many offspring you want
    IDX_CROSSOVER = 0 # Fill in at which index you want crossover
    PM = 0.5 # Mutation rate parameter

    offspring,  offspring_fitness = crossover(parents, N_OFFSPRING, IDX_CROSSOVER)
    offspring, offspring_fitness = mutation(offspring, PM)
    print("Mutation")
    print(offspring)
    print(offspring_fitness)

    # print("Mutation")
    # print(offspring_mutation)

    # Environmental selection
    curr_population = environmental_selection(curr_population, offspring)
              
# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness = calculate_fitness(curr_population)
print(fitness)
# Then return the index of that solution corresponding to the best fitness.
max_idx = np.argmin(fitness)

print("Best solution : ", curr_population[max_idx, :])
print("Best solution fitness : ", fitness[max_idx])

Generation :  0
Fitness
(10,)
[ 48.01224138 374.11049006 844.64390613 355.69244976 349.04817436
   6.89247391 914.09055645 979.22863636 469.63236481 351.57993692]
Best result in current iteration 6.892473906964563 compared to overall 6.892473906964563
Parents
(6, 2)
[[ 2.01892753  5.76780054]
 [-0.68414913  6.19066566]
 [-6.71081962 -6.80709793]
 [-6.60580809  8.52562165]
 [ 7.54561082 -2.11277514]
 [-8.98444981 -7.1639565 ]]
[  6.89247391  48.01224138 349.04817436 351.57993692 355.69244976
 374.11049006]
(array([0, 0, 1, 4, 4]), array([0, 1, 0, 0, 1]))
(5, 2)
Mutation
[[ 2.51677626  5.62631206]
 [-0.72492985  6.19066566]
 [-6.71081962 -6.80709793]
 [-6.60580809  8.52562165]
 [ 7.26212732 -1.77230504]]
[  3.08448397  48.78474027 349.04817436 351.57993692 319.77722659]
Generation :  1
Fitness
(10,)
[  3.08448397   6.89247391  48.01224138  48.78474027 319.77722659
 349.04817436 349.04817436 351.57993692 351.57993692 355.69244976]
Best result in current iteration 3.0844839661903354 compar

# Results
After running the above code, we get:
```
Best solution :  [2.99936671 4.99958592]
Best solution fitness :  5.795615705510446e-07
```
Which is quite close to the correct solution of $x=\begin{pmatrix} 3 \\ 5\end{pmatrix} $ to minimize the function $f(x) =\left(\frac{\sqrt{3}}{2} (x_1−3)+\frac{1}{2}(x_2−5)\right)^2+ 5 \cdot \left((\frac{\sqrt{3}}{2} (x_2−5)−\frac{1}{2} (x_1−3)\right)^2$

In [18]:
import sys
sys.path.append('../')
from common.GeneticAlgorithm import GeneticAlgorithm
import numpy as np
import math
PARAM_SIZE = 2
POP_SIZE = 10
LOW_BOUND = -10
HIGH_BOUND = 10

def fitness_func(total_pop):
    """ Calculates fitness using the given function f(x) = 10 *x^2
    Returns a (POP_SIZE, ) numpy array
    """       
    fitness = np.zeros(total_pop.shape[0])
    for i in range(total_pop.shape[0]):
        x_1, x_2 = total_pop[i]
        factor1 = (math.sqrt(3)/2 *(x_1-3) + 0.5*(x_2-5))**2
        factor2 = 5*(math.sqrt(3)/2 *(x_2-5) - 0.5*(x_1-3))**2
        fitness[i] = factor1+ factor2
    return fitness

In [21]:
genetic_alg = GeneticAlgorithm(
    PARAM_SIZE, 
    POP_SIZE, 
    LOW_BOUND, 
    HIGH_BOUND, 
    fitness_func
)
genetic_alg.crossover_idx = 0 # why is this the case?
genetic_alg.run(1000)

Generation : 0
Current Population: [[-3.0193834  -0.02116351]
 [ 7.23502825 -2.6333632 ]
 [ 3.63578797 -6.15726399]
 [-9.03323816  3.06761492]
 [ 1.49530679 -4.66312998]
 [-3.30430873  5.81667048]
 [-2.76741886  8.6736537 ]
 [-3.1663029   1.90350366]
 [ 0.03051362 -4.07868253]
 [-7.86118376 -2.99303238]]
Best result in current iteration 48.256412355225606 compared to overall 48.256412355225606
(array([0, 1, 2, 2, 3]), array([1, 1, 0, 1, 0]))
(4, 2)
Generation : 1
Current Population: [[-3.1663029   1.90350366]
 [-3.1663029   1.77908708]
 [-3.0193834   0.11280524]
 [-3.0193834  -0.02116351]
 [-3.32662916  5.38086144]
 [-3.30430873  5.81667048]
 [-7.86118376 -2.99303238]
 [-7.9543396  -2.99303238]
 [-2.76741886  8.6736537 ]
 [-9.03323816  3.06761492]]
Best result in current iteration 48.256412355225606 compared to overall 48.256412355225606
(array([0, 0, 1, 2, 2, 3]), array([0, 1, 0, 0, 1, 0]))
(4, 2)
Generation : 2
Current Population: [[-3.1663029   1.90350366]
 [-3.1663029   1.77908708]